In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
!echo "HOSTNAME     " `hostname`; 
!echo "CURRENT DIR  " `pwd`

# TAREA: Paraleliza el siguiente código
usando `omp for` y `omp critical` 
<br/>
juega con distintos valores de `omp for schedule(static,n)` y observa la escalabilidad

In [ ]:
%%writefile files/array_total.c
#include <stdio.h>
#include <omp.h>
#include <stdlib.h>

#define MAX_THREADS 16
#define SIZE 20000000

int main ()
{
    
    double *a = (double *)malloc(sizeof(double)*SIZE);
    int i,j;
    double start_time, run_time;

    // fill up some values
    for (i=0; i<SIZE; i++) a[i] = 1;

    printf("total secs nthreads\n");

    for (j=1;j<=MAX_THREADS ;j++) {
        omp_set_num_threads(j);
        start_time = omp_get_wtime();

        // --- ESTE ES EL CODIGO QUE TIENES QUE PARALELIZAR
        double total = 0.0;
        for (i=0; i<SIZE; i++) {
            total += a[i];
        }            
        // --- HASTA AQUI

        run_time = omp_get_wtime() - start_time;
        printf("%f %f %d\n",total, run_time,j);
    }
}

In [ ]:
!make -f m/Makefile-p2.4.1 clean all

In [ ]:
!./files/array_total > files/array_total.data
df = pd.read_csv("files/array_total.data", sep=" ")
secs     = df['secs']
nthreads = df['nthreads']
total    = df['total']
print np.mean(total)
ideal_secs = secs[0] / nthreads
ax = plt.figure(figsize=(10,5)).add_subplot(111)
ax.plot(nthreads, secs, linewidth=5, alpha=0.5)
ax.plot(nthreads, ideal_secs, linewidth=2, alpha=1., c="black")


# TAREA: Paraleliza los bucles del siguiente código
# PREGUNTA
+ Explica porqué unas operaciones tardan más que otras


In [ ]:
%%writefile files/array_sum.c
#include <stdio.h>
#include <omp.h>
#include <stdlib.h>

#define MAX_THREADS 10
#define SIZE 10000000
#define ATYPE double

int main ()
{
    
    ATYPE *a = (ATYPE *)malloc(sizeof(ATYPE)*SIZE);
    ATYPE *c = (ATYPE *)malloc(sizeof(ATYPE)*SIZE);
    int i,j;
    double start_time, run_time1, run_time2, run_time3, run_time4, run_time5;

    printf("secs1 secs2 secs3 secs4 secs5 nthreads\n");

    for (j=1;j<=MAX_THREADS ;j++) {
        omp_set_num_threads(j);
        ATYPE x = 12;

        for (i=0; i<SIZE; i++) {
            c[i] = 2;
            a[i] = 10;
        }

        start_time = omp_get_wtime();
        for (i=0; i<SIZE; i++) x = 10*i;
        run_time1 = omp_get_wtime() - start_time;

        start_time = omp_get_wtime();
        for (i=0; i<SIZE; i++) x = c[i]*10;
        run_time2 = omp_get_wtime() - start_time;

        start_time = omp_get_wtime();
        for (i=0; i<SIZE; i++) c[i] = c[i]*x;
        run_time4 = omp_get_wtime() - start_time;

        start_time = omp_get_wtime();
        for (i=0; i<SIZE; i++) c[i] = a[i]*x;
        run_time5 = omp_get_wtime() - start_time;

        start_time = omp_get_wtime();
        for (i=0; i<SIZE; i++) c[i]=x;
        run_time3 = omp_get_wtime() - start_time;

        printf("%f %f %f %f %f %d\n", run_time1,run_time2, run_time3, run_time4, run_time5, j);
    } 
}

In [ ]:
!make -f m/Makefile-p2.4.2 clean all

In [ ]:
!./files/array_sum > files/array_sum.data
df = pd.read_csv("files/array_sum.data", sep=" ")
secs1     = df['secs1']
secs2     = df['secs2']
secs3     = df['secs3']
secs4     = df['secs4']
secs5     = df['secs5']
nthreads = df['nthreads']
ideal_secs = np.min([secs1[0],secs2[0], secs3[0], secs4[0]]) / nthreads
ax = plt.figure(figsize=(10,5)).add_subplot(111)
ps1, = ax.plot(nthreads, secs1, linewidth=5, alpha=0.5)
ps2, = ax.plot(nthreads, secs2, linewidth=5, alpha=0.5)
ps3, = ax.plot(nthreads, secs3, linewidth=5, alpha=0.5)
ps4, = ax.plot(nthreads, secs4, linewidth=5, alpha=0.5)
ps5, = ax.plot(nthreads, secs5, linewidth=5, alpha=0.5)
pi, = ax.plot(nthreads, ideal_secs, linewidth=2, alpha=1., c="black")
ax.legend([ps1,ps2,ps3,ps4,ps5, pi],["1. only op" , "2. read only", "3. write only", "4. read-write", "5. cross rw", "ideal"])